# Analiza ryzyka ataków zwierząt w korelacji z gęstością zaludnienia w Australii
Autorzy: Bartosz Kołaciński 251554, Wiktor Pankanin 251606, Edyta Szymańska 251647


## Wstęp
Australia ze względu na długotrwałe odizolowanie od reszty świata ma unikalną i bogatą faunę. Jest jednym z niewielu miejsc na świecie, gdzie aktywność ludzka w wodzie często spotyka się z naturalny środowiskiem drapieżników wodnych.

Poniższy projekt odpowiada na realną potrzebę zdefiniowania czynników ryzyka tych spotkań. W dobie postępujących zmian klimatycznych, które wpływają na temperaturę wód, migrację gatunków oraz rosnącej populacji miast, zrozumienie powodu tych ataków staje się kluczowe dla bezpieczeństwa.

## Cel projektu
Głównym celem projektu jest identyfikacja i wizualizacja częstych punktów ataków krokodylii i rekinów w latach 2015 - 2024. Poszczególne cele:
- Mapowanie incydentów na tle gęstości zaludnienia
- Porówananie śmiertelności i częstotliwości ataków w zależności od gatunku
- Zbadanie sezonowości i tendów czasowych występowania zagrożeń.

## Założenia projektu
* **Zakres czasowy**: Analizujemy dane od roku 2015, aby skupić się na aktualnych trendach.
* **Zakres terytorialny**: Wyłącznie kontynent Australii i przyległe wyspy.
* **Dane demograficzne**: Używamy siatki gęstości zaludnienia (ABS 2023-24) jako przybliżenia obecności człowieka na wybrzeżu.

## Opis danych

#### Ataki krokodyli: croc_attacks.csv
Zbiór zawiera historyczne dane dotyczące incydentów z udziałem krokodyli w Australii. Dane w bazie są światowe od 2015 roku, jednak dla naszego projektu ograniczyliśmy je do terenów Australii. Źródło: https://crocattack.org/database/
- species: Gatunek krokodyla
- species_id: Czy gatunek został potwierdzony
- lat / long: Współrzędne geograficzne (szerokość i długość) miejsca ataku
- is_fatal: Zmienna określająca skutek ataku (1 = śmierć, 0 = przeżycie)
- sex: Płeć ofiary (Male = mężczyzna, Female = kobieta, Unknown = nieznana)
- date: Data ataku w formacie YYYY-MM-DD

#### Ataki rekinów: shark_attacks.xlsx
Zbiór zawiera historyczne dane dotyczące incydentów z udziałem rekinów w Australii. Dane w bazie są światowe od 1791 roku, jednak dla naszego projektu ograniczyliśmy je do terenów Australii i lat od 2015 roku. Źródło: https://zenodo.org/records/16355384
- Incident.month: Miesiąc ataku
- Incident.year: Rok ataku
- Victim.gender: Płeć ofiary
- Injury.severity: Poziom obrażeń ofiary
- Latitude / Longitude: Współrzędne geograficzne (szerokość i długość) miejsca ataku
- Shark.common.name: Nazwa gatunku rekina
- Provoked/unprovoked: Czy atak był sprowokowany

#### Gęstość zaludnienia: population_density.gpkg
Zbiór zawiera dane o gęstości zaludnienia w Australii w formacie geopackage. Źródło: https://www.abs.gov.au/statistics/people/population/regional-population/2023-24#data-downloads
- geometry: Cyfrowe granice obszarów w standardzie GDA2020.
- ERP (Estimated Resident Population): Szacunkowa liczba ludności rezydującej (dane historyczne od 2001 r. do obecnych).
- Population Density: Gęstość zaludnienia dla bieżącego roku wyrażona w osobach na kilometr kwadratowy.
- Area_sqkm: Powierzchnia obszaru w kilometrach kwadratowych.
- Components of Change: Składniki zmiany demograficznej (urodzenia, zgony, migracje wewnętrzne i zagraniczne).

### Potrzebne biblioteki
- folium : biblioteka do tworzenia interaktywnych map w formacie HTML
- geopandas : biblioteka do pracy z danymi geograficznymi
- pandas : biblioteka do analizy i manipulacji danymi
- branca : biblioteka do tworzenia kolorowych map w folium
- mathplotlib : biblioteka do tworzenia wykresów

In [ ]:
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
from branca.colormap import LinearColormap
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
from folium.plugins import HeatMapWithTime

### Wczytywanie danych

Funcja read_shark_data służy do wczytania danych z pliku excel i przetworzenia ich do GeoDataFrame.
- pobiera dane i wybiera tylko potrzebne kolumny oraz ujednolica nazwy
- tworzy nową kolumnę z pełą datą łącząć rok, miesiąc ataku
- tworzy nową kolumnę is_fatal
- ogranicza dane do lat 2015 i nowszych

In [ ]:
def read_shark_data(path: str) -> gpd.GeoDataFrame:
    df = pd.read_excel(path)

    columns = [
        "Latitude",
        "Longitude",
        "Injury.severity",
        "Victim.gender",
        "Incident.month",
        "Incident.year",
        "Shark.common.name",
        "Victim.activity",
        "Provoked/unprovoked",
    ]

    df = df[columns].copy()

    df = df.rename(
        columns={
            "Latitude": "lat",
            "Longitude": "long",
            "Victim.gender": "sex",
            "Shark.common.name": "species",
            "Victim.activity": "activity",
            "Provoked/unprovoked": "provoked",
        }
    )

    df["date"] = pd.to_datetime(
        df[["Incident.year", "Incident.month"]]
        .rename(columns={"Incident.year": "year",
                         "Incident.month": "month"})
        .assign(day=1),
        errors="coerce",
    )

    df["is_fatal"] = df["Injury.severity"].apply(
        lambda x: 1 if str(x).lower() == "fatal" else 0
    )

    df = df.dropna(subset=["lat", "long"])

    df = df[df["Incident.year"] >= 2015]

    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.long, df.lat),
        crs="EPSG:4326"
    )

    gdf = gpd.GeoDataFrame(
        gdf[
            [
                "geometry",
                "lat",
                "long",
                "is_fatal",
                "sex",
                "date",
                "species",
                "activity",
                "provoked",
            ]
        ].copy()
    )

    return gdf

Funkcja read_croc_data zajmuje się wczytywaniem danych z CSV i dostosowanie ich do wspólnego schematu nazw danych z rekinami
- ładuje dane i zamenia tekstową kolumnę z datą na datatime
- przekształca tabelę na GeoDataFrame
- ręcznie dodaje kolumnę species, activity i provoked
- wybiera i układa kolumny w ustalonej kolejności

In [ ]:
def read_croc_data(path: str) -> gpd.GeoDataFrame:
    df = pd.read_csv(path)

    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.long, df.lat),
        crs="EPSG:4326"
    )

    gdf["species"] = "crocodile"
    gdf["activity"] = None
    gdf["provoked"] = None

    gdf = gpd.GeoDataFrame(
        gdf[
            [
                "geometry",
                "lat",
                "long",
                "is_fatal",
                "sex",
                "date",
                "species",
                "activity",
                "provoked",
            ]
        ].copy()
    )

    return gdf

## Statystyki

Funkcja read_population_data odpowiada za import danych demograficznych
- konwertuje tabele na GeoDataFrame
- przelicza współrzędne na standard GPS

In [ ]:
def read_population_data(path: str) -> gpd.GeoDataFrame:
    df = pd.read_csv(path)
    pop_density_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(
        df.x, df.y))
    pop_density_gdf = pop_density_gdf.to_crs("EPSG:4326")
    return pop_density_gdf

In [ ]:
# wczytywanie danych za pomocą powyższych funkcji
data_croc = read_croc_data("../data/croc_attacks.csv")
data_shark = read_shark_data("../data/shark_attacks.xlsx")
pop_density_gdf = gpd.read_file("../data/population_density.gpkg")

# łączenie danych
all_attacks = pd.concat([data_croc, data_shark], ignore_index=True)
data_combined = gpd.GeoDataFrame(all_attacks, geometry="geometry", crs="EPSG:4326")

Generujemy tabelę porównującą ogólną liczbę atakó, liczbę ofiar śmiertelnych i wskaźnik śmiertelnośći.

In [ ]:
data_combined['animal_type'] = data_combined['species'].apply(
    lambda x: 'Krokodyl' if str(x).lower() == 'crocodile' else 'Rekin'
)

data_combined['year'] = data_combined['date'].dt.year

stats_general = data_combined.groupby('animal_type')['is_fatal'].agg(['count', 'sum', 'mean'])
stats_general.columns = ['Liczba ataków', 'W tym śmiertelne', 'Śmiertelność (%)']
stats_general['Śmiertelność (%)'] = (stats_general['Śmiertelność (%)'] * 100).round(2)
stats_general

Poniżej agregujemy dane według lat, tworząc zestawienie liczby incydentów dla każdego roku. Przedstawiamy to w formie wykresu słupkowego.

In [ ]:
stats_year = data_combined.groupby(['year', 'animal_type']).size().unstack(fill_value=0)
stats_year.index = stats_year.index.astype(int)
display(stats_year)

if not stats_year.empty:
    fig, ax = plt.subplots(figsize=(10, 5))
    stats_year.plot(kind='bar', stacked=True, color=['green', 'blue'], ax=ax)
    plt.title("Liczba ataków w Australii (2015-2024)")
    plt.xlabel("Rok")
    plt.ylabel("Liczba incydentów")
    plt.legend(title="Zwierzę")
    plt.grid(axis='y', alpha=0.3)
    plt.show()

Przypisujemy każdemu punktowi ataku wartości gęstości zaludnienia obszaru w którym wystąpił. Nastęnie obliczamy średnią gęstość zaludnienia dla miejsc ataków krokodyli i rekinów.

In [ ]:
target_crs = "EPSG:3112"
if data_combined.crs != target_crs:
    data_combined = data_combined.to_crs(target_crs)

if pop_density_gdf.crs != target_crs:
    pop_density_gdf = pop_density_gdf.to_crs(target_crs)

data_with_density = gpd.sjoin(data_combined, pop_density_gdf, how="left", predicate="within")
density_stats = data_with_density.groupby('animal_type')['pop_density_2024_people_per_km2'].mean().round(2)
density_stats

Analiza korelacji urbanistycznej: Wykorzystanie bufora 50km od centrów gęstości zaludnienia powyżej 100 os/km^2 pozwoliło na oddzielenie incydnetów "miejskich" od "dzikich"

In [ ]:
# Przygotowanie stref miejskich (bufor 50km w układzie metrycznym)
urban_zones = pop_density_gdf[pop_density_gdf['pop_density_2024_people_per_km2'] > 100].to_crs("EPSG:3112")
urban_buffers_union = urban_zones.buffer(50000).union_all()

# Przeliczanie punktów ataków na ten sam układ metryczny
attacks_metric = data_combined.to_crs("EPSG:3112")
summary_data = []

# Lista gatunków do analizy
species_list = attacks_metric['animal_type'].unique()
for animal in species_list:
    # Filtrowanie danych dla konkretnego gatunku
    subset = attacks_metric[attacks_metric['animal_type'] == animal]

    # Sprawdzanie, które ataki danego gatunku są w strefie 50km
    near_urban_mask = subset.geometry.within(urban_buffers_union)
    count_near = near_urban_mask.sum()
    total_animal = len(subset)
    percentage = (count_near / total_animal) * 100 if total_animal > 0 else 0

    summary_data.append({
        "Gatunek": animal,
        "Liczba ataków przy miastach": count_near,
        "Wszystkie ataki": total_animal,
        "Procent (%)": round(percentage, 2)
    })

# Obliczanie statystyki zbiorczej
total_near = attacks_metric.geometry.within(urban_buffers_union).sum()
total_all = len(attacks_metric)
total_pct = (total_near / total_all) * 100

summary_data.append({
    "Gatunek": "Razem",
    "Liczba ataków przy miastach": total_near,
    "Wszystkie ataki": total_all,
    "Procent (%)": round(total_pct, 2)
})

# Prezentacja wyników w formie tabeli
df_summary = pd.DataFrame(summary_data)
display(df_summary)

## Tworzenie map

Ta komórka odpowiada za wygenerowanie interaktywnej mapy HTML. Łączy ona trzy warstwy informacyjne: incydenty z udziałem zwierząt, dane demograficzne oraz analizę występowania zjawiska (heatmap). Kluczowe elementy:
- Inicjalizacja i warstwy podkładowe: mapa jest wycentrowana na Australię. Użytkownik ma trzy podkłady (Esri Satellite, OpenStreetMap, CartoDB Positron).
- Grupowanie danych: Dane są podzielone na warstwy (ataki rekinów i krokodyli z podziałem na śmietelne/nieśmierelne, gęstość zaludnienaia i heatmapa pokazująca zależność pomiędzy ilością ataków a gęstością zaludnienia).
- Wizualizacja markerów: Kolor tła punktu ataku zależy od skutku (czerwony: śmierć, pomarańczowy: obrażenia), a ikona od gatunku zwierzęcia
- Wizualizacja demograficzna: Na mapę przenoszone są dane o populacji, zastosowana jest skala logarytmiczna.
- Mapa ciepła: warstwa pokazująca zagęszczenie punktów ataków.

In [ ]:
output_file = "mapa_australia.html"

# Tworzenie mapy bazowej
m = folium.Map(
    location=[-25.2744, 133.7751],
    zoom_start=4,
    tiles=None,
)
# Dodanie różnych warstw podkładowych
folium.TileLayer(
    tiles="Esri.WorldImagery",
    name="Esri Satellite",
    control=True
).add_to(m)

folium.TileLayer(
    tiles="OpenStreetMap",
    name="OpenStreetMap",
    control=True
).add_to(m)

folium.TileLayer(
    tiles="cartodbpositron",
    name="CartoDB Positron",
    control=True
).add_to(m)

# Grupowanie danych na warstwy
fg_croc_fatal = folium.FeatureGroup(name="Ataki krokodyli: Śmiertelne")
fg_croc_non_fatal = folium.FeatureGroup(name="Ataki krokodyli: Nieśmiertelne")
fg_shark_fatal = folium.FeatureGroup(name="Ataki rekinów: Śmiertelne")
fg_shark_non_fatal = folium.FeatureGroup(name="Ataki rekinów: Nieśmiertelne")
fg_population = folium.FeatureGroup(name="Gęstość zaludnienia")
fg_heatmap_density = folium.FeatureGroup(name="Heatmap: ataki vs gęstość zaludnienia", show=False)

# Ścieżka do plików z ikonami
croc_icon_path = "../icons/crocodile.png"
shark_icon_path = "../icons/shark.png"

# Poprawienie crs
data_combined = data_combined.to_crs("EPSG:4326")

# Dodawania markerów ataków na mapie
for _, row in data_combined.iterrows():
    popup_info = ""
    for col in data_combined.columns:
        if col != "geometry":
            value = row[col]
            if pd.notna(value) and value != "" and value is not None:
                popup_info += f"<b>{col}:</b> {value}<br>"
    # Rozróżnienie gatunków
    is_shark = "shark" in str(row.get("species", "")).lower()
    icon_path = shark_icon_path if is_shark else croc_icon_path

    # Dodawanie odpowiednych kolorów
    if row["is_fatal"] == 1:
        bg_color = "#d32f2f"
        target_group = fg_shark_fatal if is_shark else fg_croc_fatal
    else:
        bg_color = "#f57c00"
        target_group = fg_shark_non_fatal if is_shark else fg_croc_non_fatal

    # Tworzenie własnej ikony markeru
    icon_html = f"""
        <div style="background-color: {bg_color}; border-radius: 50%; width: 24px; height: 24px;
                    display: flex; justify-content: center; align-items: center; border: 2px solid white;
                    box-shadow: 0 0 5px rgba(0,0,0,0.5);">
            <img src="{icon_path}" style="width: 16px; height: 16px;">
        </div>"""

    # Dodawanie markerów do odpowiedniej grupy
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=folium.Popup(popup_info, max_width=300),
        icon=folium.DivIcon(html=icon_html, icon_size=(24, 24), icon_anchor=(12, 12))
    ).add_to(target_group)

# Wizualizacja danych demograficznych
if pop_density_gdf is not None and not pop_density_gdf.empty:
    pop = pop_density_gdf.copy().to_crs("EPSG:4326")

    possible_names = ["pop_density_2024_people_per_km2", "population_density", "pop_density", "density", "value"]
    field = next((n for n in possible_names if n in pop.columns), pop.select_dtypes(include=['number']).columns[0])

    # Przygotowanie skali kolorów
    vals = pop[field].dropna()
    vals_log = np.log1p(vals.astype(float))
    log_min, log_max = float(vals_log.min()), float(vals_log.max())
    scale_min, scale_max = 0, float(vals.max())
    cmap = LinearColormap(
        ["#ffffb2", "#fecc5c", "#fd8d3c", "#f03b20", "#bd0026"],
        vmin=int(scale_min), vmax=int(scale_max),
        caption="Gęstość zaludnienia (os./km²)"
    )

    # Funkcja pomocnicza mapująca wartość na konkretny kolor
    def _density_to_color(v, cmap):
        if v is None or (isinstance(v, float) and np.isnan(v)):
            return "#ffffff00"
        pos = (np.log1p(float(v)) - log_min) / (log_max - log_min)
        pos = np.clip(pos, 0, 1)
        return cmap(scale_min + pos * (scale_max - scale_min))

    # Dodawanie danych demograficznych na mapie
    geom_types = set(pop.geometry.geom_type)
    if any("polygon" in gt.lower() for gt in geom_types):
        pop['geometry'] = pop['geometry'].simplify(0.01)

        folium.GeoJson(
            pop.to_json(),
            style_function=lambda feature: {
                "fillColor": _density_to_color(feature["properties"].get(field), cmap),
                "color": "black", "weight": 0, "fillOpacity": 0.5
            },
            name="Gęstość zaludnienia (obszary)"
        ).add_to(fg_population)
    else:
        for _, prow in pop.iterrows():
            centroid = prow.geometry.centroid
            folium.CircleMarker(
                location=[centroid.y, centroid.x],
                radius=5, fill=True, fill_color=_density_to_color(prow[field], cmap),
                color=None, fill_opacity=0.7,
                popup=f"{field}: {prow[field]:.0f} os./km²"
            ).add_to(fg_population)

    # Dodanie legendy kolorów do mapy
    cmap.add_to(m)

# Tworzenie mapy ciepła
heatmap_data = [[row.geometry.y, row.geometry.x, 1.0] for _, row in data_combined.iterrows()]
if pop_density_gdf is not None and not pop_density_gdf.empty and 'pop' in dir() and 'field' in dir():
    attacks_with_density = gpd.sjoin(data_combined, pop[[field, "geometry"]], how="left", predicate="within")
    heatmap_data = [[row.geometry.y, row.geometry.x, float(row[field]) if pd.notna(row.get(field)) else 1.0] for _, row in attacks_with_density.iterrows()]

# Dodanie warstwy mapy ciepła z gradientem
HeatMap(heatmap_data, radius=25, blur=20, min_opacity=0.3, max_zoom=14,
        gradient={"0.2": "blue", "0.4": "cyan", "0.6": "lime", "0.8": "yellow", "1.0": "red"}).add_to(fg_heatmap_density)
fg_heatmap_density.add_to(m)

# Dodanie wszystkich grup do mapy
fg_croc_fatal.add_to(m)
fg_croc_non_fatal.add_to(m)
fg_shark_fatal.add_to(m)
fg_shark_non_fatal.add_to(m)
fg_population.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

# Zapisywanie mapy i wyświetlenie
m.save(output_file)
m

### Mapa ciepła ataków

Mapa przedstawiająca mapę ciepła ataków (z warstwą przedstawiającą gęstość zaludnienia)

In [ ]:
nowa_mapa = folium.Map(
    location=[-25.2744, 133.7751],
    zoom_start=4,
    tiles="cartodbpositron"
)

folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Widok Satelitarny',
    overlay=False
).add_to(nowa_mapa)

points = [[row.geometry.y, row.geometry.x] for index, row in data_combined.iterrows()]

fg_heatmap = folium.FeatureGroup(name="Mapa Ciepła Ataków", show=True)
HeatMap(
    data=points,
    radius=25,
    blur=20,
    min_opacity=0.4,
    gradient={"0.2": 'blue', "0.4": 'cyan', "0.6": 'lime', "0.8": 'yellow', "1.0": 'red'}
).add_to(fg_heatmap)
fg_heatmap.add_to(nowa_mapa)

if 'pop_density_gdf' in locals() and pop_density_gdf is not None:
    fg_pop = folium.FeatureGroup(name="Gęstość zaludnienia (kontury)", show=False)
    pop = pop_density_gdf.copy().to_crs("EPSG:4326")

    if any("polygon" in gt.lower() for gt in set(pop.geometry.geom_type)):
         pop['geometry'] = pop['geometry'].simplify(0.05)

    folium.GeoJson(
        pop.to_json(),
        style_function=lambda x: {
            'fillColor': '#4B0082',
            'color': '#2E0854',
            'weight': 0.5,
            'fillOpacity': 0.1
        }
    ).add_to(fg_pop)
    fg_pop.add_to(nowa_mapa)

folium.LayerControl(collapsed=False).add_to(nowa_mapa)

nowa_mapa

#### Mapa pokazująca temperaturę wody oraz miejsca ataków
Analiza środowiskowa, integruje dane wektorowe z danymi rastrowymi, które są dynamicznie pobierane z zewnętrznych serwerów. Kluczowe elementy:
- Integracja WMS: Implementacja protokołu OGC WMS w celu pobrania warstwy Sea Surface Temperature bezpośrodnie z serwerów NASA GIBS.
- Przetwarzanie danych: Nanoszenie interaktywnych markerów

In [ ]:
# Tworzenie mapy bazowej, ustawienie jej na środek Australii i zastosowanie trybu ciemnego mapy
mapa_sst = folium.Map(
    location=[-25.2744, 133.7751],
    zoom_start=4,
    tiles="cartodbdark_matter"
)

# Konfiguracja warstwy WMS do pobierania danych z NASA Earth
folium.WmsTileLayer(
    url="https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi",
    layers="GHRSST_L4_MUR_Sea_Surface_Temperature",
    name="Temperatura wody (SST)",
    fmt="image/png",
    transparent=True,
    overlay=True,
    attr="NASA EarthData"
).add_to(mapa_sst)

# Utworzenie grupy warstw
fg_ataki_ikony = folium.FeatureGroup(name="Ikony ataków (Szczegóły)", show=True)

# Definicja ścieżek do ikon
croc_icon_path = "icons/crocodile.png"
shark_icon_path = "icons/shark.png"

# Generowanie warstwy wektorowej
for _, row in data_combined.iterrows():
    popup_info = "".join([f"<b>{col}:</b> {row[col]}<br>" for col in data_combined.columns if col != "geometry" and pd.notna(row[col])])

    is_shark = "shark" in str(row.get("species", "")).lower()
    icon_p = shark_icon_path if is_shark else croc_icon_path
    bg_color = "#d32f2f" if row["is_fatal"] == 1 else "#f57c00"

    icon_html = f"""
        <div style="background-color: {bg_color}; border-radius: 50%; width: 22px; height: 22px;
                    display: flex; justify-content: center; align-items: center; border: 2px solid white;
                    box-shadow: 0 0 5px rgba(0,0,0,0.5);">
            <img src="{icon_p}" style="width: 14px; height: 14px;">
        </div>"""

    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=folium.Popup(popup_info, max_width=300),
        icon=folium.DivIcon(html=icon_html, icon_size=(22, 22), icon_anchor=(11, 11))
    ).add_to(fg_ataki_ikony)

# Dodanie grupy warstwy do mapy
fg_ataki_ikony.add_to(mapa_sst)

# Opis legendy
legend_html = """
<div style="
    position: fixed;
    bottom: 50px; left: 50px; width: 200px; height: 280px;
    background-color: rgba(255, 255, 255, 0.9); color: black; border:2px solid grey; z-index:9999; font-size:13px;
    padding: 15px; border-radius: 12px; font-family: 'Arial', sans-serif;
    ">
    <p style="margin-top: 0; margin-bottom: 10px;"><b>Skala temperatury wody</b></p>

    <div style="
        background: linear-gradient(to top, #0000ff, #00ffff, #00ff00, #ffff00, #ff0000);
        width: 30px; height: 150px; float: left; border: 1px solid #333; margin-right: 15px;">
    </div>

    <div style="line-height: 30px; height: 150px; display: flex; flex-direction: column; justify-content: space-between;">
        <span><b>32°C</b> (Gorąca)</span>
        <span><b>24°C</b> (Ciepła)</span>
        <span><b>16°C</b> (Chłodna)</span>
        <span><b>8°C</b> (Zimna)</span>
    </div>

    <div style="clear: both; padding-top: 15px; border-top: 1px solid #ccc; margin-top: 10px;">
        <div style="margin-bottom: 5px;">
            <span style="color: #d32f2f; font-size: 18px;">●</span> Atak śmiertelny
        </div>
        <div>
            <span style="color: #f57c00; font-size: 18px;">●</span> Atak nieśmiertelny
        </div>
    </div>
</div>
"""

# Iniekcja elementu HTML do mapy
mapa_sst.get_root().html.add_child(folium.Element(legend_html))

# Dodanie kontrolera warstw
folium.LayerControl(collapsed=True).add_to(mapa_sst)

# Wyświetlenie mapy
mapa_sst

Mapa przedstawiająca zmianę ataków w czasie na podstawie danych wektorowych.
- Agregacja czasowa: Tworzy listę współrzędnych [lat, lon] pogrupowaną według lat (od 2015 do 2024).
- Dynamiczna wizualizacja: Wykorzystuje wtyczkę HeatMapWithTime do nałożenia animowanej warstwy zagęszczenia ataków.
- Interaktywność: Dodaje suwak czasu (Time Slider), który pozwala na ręczne lub automatyczne śledzenie zmian lokalizacji "gorących punktów" (hot-spots) na przestrzeni dekady.

In [ ]:
# Przygotowanie czystych danych
data_for_heat = data_combined.copy().to_crs("EPSG:4326")
data_for_heat = data_for_heat.dropna(subset=['year', 'geometry'])

# Pobieramy lata
years = sorted(data_for_heat['year'].unique())
heat_data = []

for year in years:
    # Filtrowanie dla danego roku
    year_subset = data_for_heat[data_for_heat['year'] == year]
    points = year_subset.geometry.apply(lambda geom: [float(geom.y), float(geom.x)]).tolist()
    heat_data.append(points)

# Inicjalizacja mapy
map_trends = folium.Map(location=[-25.2744, 133.7751], zoom_start=4, tiles="cartodbpositron")

# Konfiguracja HeatMapWithTime
# Jeśli punkty są "za małe", zwiększamy 'radius'
HeatMapWithTime(
    data=heat_data,
    index=[str(int(y)) for y in years],
    radius=25,
    min_opacity=0.3,
    max_opacity=0.8,
    auto_play=False,
    use_local_extrema=True
).add_to(map_trends)

map_trends

## Wnioski
#### Zależność między atakami a gęstością zaludnienia
Ataki rekinów wskazują silną korelację z gęstością zaludnienia, sugeruje to, że ataki rekinów są zdarzeniami losowymi wynikającymi z dużej liczby ludzi w wodzie.
Incydenty z krokodylami występują głównie w obszarach o skrajnie niskiej gęstości zaludnienia (dzika północ Australii), co wskazuje na terytorialny charakter tych zwierząt.
#### Analiza 50 km od miast
Ponad 60% wszystkich ataków ma miejsce w strefie 50 km od miast. Pozwala to na sformuowanie wniosku dla służb ratunkowych: potrzebne są tam kampanie edukacyjne i systemy monitoringu.
#### Wnioski statystyczne
Obszaty średniej gęstości zaludnienia ale intensywnej turystyce, wykazują nieproporcjonalnie wysoką liczbą ataków w stosunku do stałych mieszkańców.